In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
from time import sleep

In [2]:
# Get the Wuzzuf sitemap URL
sitemap_url = "https://wuzzuf.net/sitemap-job-1.xml"
response = requests.get(sitemap_url)

# Parse the XML
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "xml")
    urls = [loc.text for loc in soup.find_all("loc")]
        
else:
    print("Failed to fetch the sitemap.")

In [4]:
len(urls)

8595

In [5]:
title = []
working_hours = []
working_place = []
company = []
location = []
post_date = []
num_of_applicants = []
num_of_positions = []
experience = []
career_level = []
education = []
salary = []
job_category = []
skills = []
job_description = []
job_requirements = []

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(urls[1], headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
doc = lh.fromstring(response.content)

# Extract each data point using xpath
print(doc.xpath('//section/div/h1')[0].text_content().strip())
print(doc.xpath('//section/div/div/div/a/span')[0].text_content().strip())
print(doc.xpath('//section/div/div/a/span')[0].text_content().strip())
print(doc.xpath('//strong/div/a')[0].text_content().strip())
print(doc.xpath('//strong')[0].text_content().strip().split('-')[-1].strip())
print(doc.xpath('//section/div/span')[0].text_content().strip())
print(doc.xpath('//section/div/div/div/strong')[0].text_content().strip())
print(doc.xpath('//div/div/span/span')[1].text_content().strip())
print(doc.xpath('//main//section/div/span/span')[0].text_content().strip())
# print(doc.xpath('//div[@class="career-level"]')[0].text_content().strip())
# print(doc.xpath('//div[@class="education"]')[0].text_content().strip())
# print(doc.xpath('//div[@class="salary"]')[0].text_content().strip())
# print(doc.xpath('//div[@class="category"]')[0].text_content().strip())
# print(doc.xpath('//div[@class="skills"]')[0].text_content().strip())
# print(doc.xpath('//div[@class="description"]')[0].text_content().strip())
# print(doc.xpath('//div[@class="requirements"]')[0].text_content().strip())


Senior Backend Developer
Full Time
Hybrid
RocketDevs
Cairo, Egypt
Posted 27 days ago
12
1 open position


IndexError: list index out of range